### Zid Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from cv2.ximgproc import guidedFilter
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import sys

from utils.collection import HazeCollection
from utils.network import VaeNet
from utils.skip import Skip
from utils.loss import StdLoss
from utils.utils import get_atmosphere, torch_to_np, np_to_torch, save_image, get_information
from utils.imresize import np_imresize

#### Collection

In [2]:
haze_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/synthetic"
image_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/original"

haze_path_dt2 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/real-world"

haze_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/hazy_val"
image_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/gt_val"

In [3]:
size = 256
channels = 3

collection_dt1 = HazeCollection(haze_path_dt1, image_path_dt1)
collection_dt2 = HazeCollection(haze_path_dt2)
collection_dt3 = HazeCollection(haze_path_dt3, image_path_dt3)

#### Model

In [4]:
# Skip
un = Skip(
    channels, 3,
    num_channels_down=[8, 16, 32, 64, 128],
    num_channels_up=[8, 16, 32, 64, 128],
    num_channels_skip=[0, 0, 0, 4, 4],
    upsample_mode='bilinear',
    need_sigmoid=True, need_bias=True, pad='reflection'
)

x = torch.zeros([1, channels, size, size])
a = un(x)
a.shape

torch.Size([1, 3, 256, 256])

#### Training

In [5]:
class ZidTrainer:
    def __init__(
        self,
        metrics=[]
    ):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.metrics = metrics

    def init(self, image):
        input_depth = 3
        pad = 'reflection'
        netJ = Skip(
            input_depth, 3,
            num_channels_down=[8, 16, 32, 64, 128],
            num_channels_up=[8, 16, 32, 64, 128],
            num_channels_skip=[0, 0, 0, 4, 4],
            upsample_mode='bilinear',
            need_sigmoid=True, need_bias=True, pad=pad
        )

        netT = Skip(
            input_depth, 1,
            num_channels_down=[8, 16, 32, 64, 128],
            num_channels_up=[8, 16, 32, 64, 128],
            num_channels_skip=[0, 0, 0, 4, 4],
            upsample_mode='bilinear',
            need_sigmoid=True, need_bias=True, pad=pad
        )

        self.model = {
            "netJ": netJ.to(self.device),
            "netT": netT.to(self.device),
            "netA": VaeNet(image.shape).to(self.device)
        }
        self.loss = {
            "mse": nn.MSELoss().to(self.device),
            "std": StdLoss().to(self.device)
        }

        atmosphere = get_atmosphere(image)
        self.valA = nn.Parameter(data=torch.cuda.FloatTensor(
            atmosphere.reshape((1, 3, 1, 1))), requires_grad=False)

        parameters = [p for p in self.model["netJ"].parameters()]
        parameters += [p for p in self.model["netT"].parameters()]
        parameters += [p for p in self.model["netA"].parameters()]

        self.optimizer = optim.Adam(
            parameters,
            lr=self.learning_rate
        )

    def t_matting(self, mask_out_np, original):
        refine_t = guidedFilter(original.transpose(1, 2, 0).astype(
            np.float32), mask_out_np[0].astype(np.float32), 50, 1e-4)
        return np.array([np.clip(refine_t, 0.1, 1)])

    def train(self, information, epochs):
        (haze, image, name) = information
        self.init(haze)
        loop = tqdm(range(epochs))

        losses = []
        lossFinal = None

        hazeTorch = np_to_torch(haze)
        hazeTorch = hazeTorch.to(device=self.device)
        for i in loop:
            # forward
            predT = self.model["netT"](hazeTorch)
            predJ = self.model["netJ"](hazeTorch)
            predA = self.model["netA"](hazeTorch)

            lossT = self.loss["std"](predT)
            lossJ = self.loss["mse"](predT * predJ + (1 - predT) * predA, hazeTorch)
            lossA = self.model["netA"].getLoss()

            dcp_prior = torch.min(predJ.permute(0, 2, 3, 1), 3)[0]
            dcp_loss = self.loss["mse"](dcp_prior, torch.zeros_like(dcp_prior)) - 0.05

            lossFinal = lossJ + lossA
            lossFinal += 0.005 * lossT
            lossFinal += dcp_loss
            
            lossFinal += 0.1 * self.loss["std"](predA)
            lossFinal += self.loss["mse"](predA, self.valA * torch.ones_like(predA))

            #if i % 100 == 0 or i == epochs-1:
            if i == epochs-1:
                imageJ = np.clip(torch_to_np(predJ), 0, 1)
                imageT = np.clip(torch_to_np(predT), 0, 1)
                imageA = np.clip(torch_to_np(predA), 0, 1)
                imageT = self.t_matting(imageT, haze)

                # I(x) = J(x)t(x) + A(1 − t(x))
                post = np.clip((haze - ((1 - imageT) * imageA)) / imageT, 0, 1)
                post = np_imresize(post, output_shape=haze.shape[1:])
                save_image('{}_{}'.format(name[0].split('.')[0], i+1), post, './images/zid/' + self.dt_number)

            # backward
            self.optimizer.zero_grad()
            lossFinal.backward(retain_graph=True)
            self.optimizer.step()

            # update tqdm
            loop.set_postfix(loss=lossFinal.item())
            losses.append(lossFinal.item())

        # metrics
        finalSSIM, finalPSNR = 0, 0
        if image is not None:
            finalSSIM = ssim(post, image, channel_axis=0, data_range=1)
            finalPSNR = psnr(post, image)
            print("Ssim: {:.4f}, Psnr: {:.4f}\n".format(finalSSIM, finalPSNR))

        return [np.sum(losses)/len(losses), finalSSIM, finalPSNR]

    def fit(
        self,
        collection,
        epochs=1,
        dt_number='',
        learning_rate=1e-3
    ):
        historial = []
        self.dt_number = dt_number
        self.learning_rate = learning_rate
        #loop = enumerate(tqdm(collection))
        for i, info in enumerate(collection):
            print('Image {}/{} [{}]'.format(i+1, len(collection), info[2][0]))
            loss, ssim, psnr = self.train(info, epochs)
            historial.append([loss, ssim, psnr])
        return historial

In [6]:
dehazing = ZidTrainer()

#### Testing

In [7]:
epochs = 512
learning_rate = 1e-4

##### D1

In [8]:
historial_dt1 = dehazing.fit(collection_dt1, epochs, 'd1/', learning_rate)
get_information(historial_dt1, collection_dt1)

Image 1/10 [1381.jpg]


100%|██████████| 512/512 [00:33<00:00, 15.36it/s, loss=0.0157]


Ssim: 0.6570, Psnr: 12.5898

Image 2/10 [5576.jpg]


100%|██████████| 512/512 [00:26<00:00, 19.68it/s, loss=0.0397]


Ssim: 0.6823, Psnr: 17.0888

Image 3/10 [7471.jpg]


...

Ssim: 0.9148, Psnr: 22.2337

Image 9/10 [1352.jpg]


100%|██████████| 512/512 [01:08<00:00,  7.47it/s, loss=0.00612]


Ssim: 0.7347, Psnr: 14.7210

Image 10/10 [4561.jpg]


100%|██████████| 512/512 [00:26<00:00, 19.14it/s, loss=0.0453]

Ssim: 0.7668, Psnr: 20.1741

Mean SSIM: 0.7582136929035187
Mean PSNR: 17.078968472650416
Best SSIM: 8180.jpg
Best PSNR: 8180.jpg


##### D2

In [9]:
historial_dt2 = dehazing.fit(collection_dt2, epochs, 'd2/', learning_rate)
get_information(historial_dt2, collection_dt2)

Image 1/10 [SFC_Google_197.jpeg]


  0%|          | 0/512 [00:00<?, ?it/s]

100%|██████████| 512/512 [00:50<00:00, 10.10it/s, loss=0.014] 


Image 2/10 [MLS_Bing_117.jpeg]


100%|██████████| 512/512 [00:57<00:00,  8.88it/s, loss=0.0212]


...

Image 9/10 [KRO_Google_143.jpeg]


100%|██████████| 512/512 [00:56<00:00,  9.00it/s, loss=0.0262]


Image 10/10 [HazeDr_Google_404.jpeg]


100%|██████████| 512/512 [02:02<00:00,  4.17it/s, loss=0.103]

Mean SSIM: 0.0
Mean PSNR: 0.0
Best SSIM: SFC_Google_197.jpeg
Best PSNR: SFC_Google_197.jpeg


##### D3

In [10]:
historial_dt3 = dehazing.fit(collection_dt3, epochs, 'd3/', learning_rate)
get_information(historial_dt3, collection_dt3)

Image 1/10 [1410_5.png]


  1%|          | 4/512 [00:00<00:40, 12.44it/s, loss=0.77] 

100%|██████████| 512/512 [00:43<00:00, 11.64it/s, loss=0.0489]


Ssim: 0.7944, Psnr: 16.1664

Image 2/10 [1430_5.png]


100%|██████████| 512/512 [00:43<00:00, 11.74it/s, loss=-.00943] 


...

Ssim: 0.8132, Psnr: 18.4693

Image 9/10 [1425_5.png]


100%|██████████| 512/512 [00:43<00:00, 11.72it/s, loss=0.00812]


Ssim: 0.7469, Psnr: 16.7833

Image 10/10 [1420_5.png]


100%|██████████| 512/512 [00:43<00:00, 11.78it/s, loss=0.0288]


Ssim: 0.8229, Psnr: 16.6879

Mean SSIM: 0.7783678829669952
Mean PSNR: 17.839021457437095
Best SSIM: 1405_5.png
Best PSNR: 1405_5.png
